# Autograd: 자동 미분

- Autograd 패키지는 tensor의 모든 연산에 자동 미분을 제공합니다. 이는 define-by-run의 프레임워크로 코드를 어떻게 작성하느냐에 따라 역전파가 정의된다는 뜻입니다. 역전파는 학습과정의 매 단계마다 달라집니다.
- .requires_grad 속성을 True로 설정하면 해당 tensor의 모든 연산을 추적합니다. 계산이 완료된 후 .backward()를 호출해 gradient를 자동으로 계산할 수 있습니다. 이 tensor의 gradient는 .grad에 누적됩니다.
- 연산 기록을 추적하는 것을 멈추기 위해 코드 블럭을 with torch.no_grad():로 감쌀 수 있습니다. gradient는 필요 없지만 requires_grad=True가 설정되어 학습 가능한 Parameter(매개변수)를 갖는 모델을 평가할 때 유용합니다.

In [1]:
import torch

In [2]:
x = torch.ones(2,2,requires_grad=True) 
#tensor를 생성하고 requires_grad=True로 연산을 기록합니다.
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [3]:
y = x+2 #gradient function이 자동으로 포함됩니다.
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [4]:
z = y*y*3
out = z.mean() # vector값을 scalar로 변경한 거 같은데, 실제로도 이런 식으로 vector를 scalar로 변경하나?
print(z,out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward1>)


In [7]:
a = torch.randn(2,2)
a = ((a*3)/(a-1)) 
print(a.requires_grad)
print(a.grad_fn) # 사용자가 만든 텐서의 grad_fn은 none입니다.
a.requires_grad_(True)
print(a.requires_grad) # 설정 안 했다가 True로 설정하면 True로 잡히기 시작함.
b = (a*a).sum()
print(b.grad_fn) #requires_grad_(True)로 지정하고 연산하면 이렇게 grad_fn가 생깁니다.

False
None
True


### Gradient

In [8]:
print(out) # out = 3(x+2)*2
out.backward()

tensor(27., grad_fn=<MeanBackward1>)


In [9]:
print(out.backward())

RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.

In [10]:
print(x)
print(x.grad) # d(out)/dx 를 출력합니다. -> 왜 4.5?

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


In [11]:
x = torch.randn(3,requires_grad=True)

y=x*2

while y.data.norm() < 1000:
    
    #data.norm()은 점들 사이의 유클리디안 거리를 나타냅니다
    #torch.sqrt(torch.sum(torch.pow(y, 2)))
    
    y = y*2
    
    print(y,y.data.norm())

tensor([ 1.5549, -5.6909,  2.0553], grad_fn=<MulBackward0>) tensor(6.2472)
tensor([  3.1097, -11.3818,   4.1106], grad_fn=<MulBackward0>) tensor(12.4945)
tensor([  6.2194, -22.7635,   8.2213], grad_fn=<MulBackward0>) tensor(24.9890)
tensor([ 12.4389, -45.5271,  16.4425], grad_fn=<MulBackward0>) tensor(49.9779)
tensor([ 24.8777, -91.0541,  32.8850], grad_fn=<MulBackward0>) tensor(99.9559)
tensor([  49.7554, -182.1082,   65.7701], grad_fn=<MulBackward0>) tensor(199.9118)
tensor([  99.5109, -364.2164,  131.5402], grad_fn=<MulBackward0>) tensor(399.8235)
tensor([ 199.0218, -728.4329,  263.0804], grad_fn=<MulBackward0>) tensor(799.6470)
tensor([  398.0435, -1456.8657,   526.1607], grad_fn=<MulBackward0>) tensor(1599.2941)


In [17]:
gradients = torch.tensor([0.1,1.0,0.0001],dtype=torch.float)
print(y)
y.backward(gradients)
print(x.grad) # d(y)/d(x) 를 출력합니다

tensor([  398.0435, -1456.8657,   526.1607], grad_fn=<MulBackward0>)
tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])


이건 의미가 뭐지

In [18]:
print(x.requires_grad)
print((x**2).requires_grad)

with torch.no_grad():
    print((x**2).requires_grad) #tensor들의 연산 기록 추적을 막을 수 있습니다.
     
print((x**2).requires_grad)

True
True
False
True


# MNIST

In [20]:
import torch.nn as nn
import torch.optim as optim

from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets

batch_size = 32
learning_rate = 0.01
num_epochs = 5

train_dataset = datasets.MNIST(root = './data', train = True, transform = transforms.ToTensor())
test_dataset = datasets.MNIST(root = './data',train = False, transform = transforms.ToTensor())

In [22]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=batch_size,shuffle=False)

In [24]:
class Neuralnetwork(nn.Module):
    def __init__(self, num_classes = 10):
        super(Neuralnetwork, self).__init__()
        self.layer1 = nn.Linear(28*28, 100)
        self.layer2 = nn.Linear(100,200)
        self.layer3 = nn.Linear(200, num_classes)
        
    def forward(self, x):
        out = x.view(x.size(0),-1) # x.size는 batch size다. 
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        return out

In [25]:
model = Neuralnetwork()

In [26]:
criterion = nn.CrossEntropyLoss()

In [27]:
optimizer = optim.SGD(model.parameters(), lr = learning_rate)

In [30]:
for epoch in range(num_epochs):
    for idx, (img, label) in enumerate(train_loader,1):
        img, label = Variable(img), Variable(label)
        out = model(img)
        loss = criterion(out,label)
        
        # backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (idx+1) % 100 ==0:
            print("Epoch [{}/{}], Step [{}/{}], Loss : {:.4f}".format(epoch+1, num_epochs, idx+1, len(train_loader), loss.item()))
            

Epoch [1/5], Step [100/1875], Loss : 1.8111
Epoch [1/5], Step [200/1875], Loss : 1.2558
Epoch [1/5], Step [300/1875], Loss : 1.1683
Epoch [1/5], Step [400/1875], Loss : 0.6202
Epoch [1/5], Step [500/1875], Loss : 0.6859
Epoch [1/5], Step [600/1875], Loss : 0.7337
Epoch [1/5], Step [700/1875], Loss : 0.6719
Epoch [1/5], Step [800/1875], Loss : 0.6064
Epoch [1/5], Step [900/1875], Loss : 0.3513
Epoch [1/5], Step [1000/1875], Loss : 0.3808
Epoch [1/5], Step [1100/1875], Loss : 0.5986
Epoch [1/5], Step [1200/1875], Loss : 0.6180
Epoch [1/5], Step [1300/1875], Loss : 0.2590
Epoch [1/5], Step [1400/1875], Loss : 0.4708
Epoch [1/5], Step [1500/1875], Loss : 0.3779
Epoch [1/5], Step [1600/1875], Loss : 0.4954
Epoch [1/5], Step [1700/1875], Loss : 0.3179
Epoch [1/5], Step [1800/1875], Loss : 0.3693
Epoch [2/5], Step [100/1875], Loss : 0.4486
Epoch [2/5], Step [200/1875], Loss : 0.4823
Epoch [2/5], Step [300/1875], Loss : 0.2172
Epoch [2/5], Step [400/1875], Loss : 0.1886
Epoch [2/5], Step [500/

In [31]:
model.eval()

correct = 0
total = 0
for img, label in test_loader:
    out = model(img)
    _, predicted = torch.max(out.data,1)
    total += label.size(0)
    correct+= (predicted == label).sum().item()


print('Test Accuracy of the Model on the 10000 test images : {} %'.format(100 * correct / total))

Test Accuracy of the Model on the 10000 test images : 92.13 %


In [32]:
torch.save(model.state_dict(),"model_nn.ckpt")